In [52]:
import numpy as np
import pandas as pd  
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [53]:
News =pd.read_csv('Train.csv')
News.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
News.shape

(20800, 5)

In [55]:
News.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [56]:
### Drop Nan Values because it has too many missing values
News = News.dropna()

In [57]:
## Get the Independent Features

X=News.drop('label',axis=1)

In [58]:
## Get the Dependent features
y=News['label']

In [59]:
X.shape

(18285, 4)

In [60]:
y.shape

(18285,)

In [61]:
# Get the count of each unique value in the y array
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [62]:
# Import necessary modules from TensorFlow and Keras for building and training an LSTM model

# Embedding layer: converts positive integer indices into dense vectors of fixed size
from tensorflow.keras.layers import Embedding  # for converting categorical data into dense vectors

# pad_sequences: ensures all sequences in a list have the same length by padding them with zeros
from tensorflow.keras.preprocessing.sequence import pad_sequences  # for padding sequences to a fixed length

# Sequential model: a linear stack of layers in Keras
from tensorflow.keras.models import Sequential  # for building a linear stack of layers

# one_hot: encodes text into a sequence of integers
from tensorflow.keras.preprocessing.text import one_hot  # for encoding text data into numerical format

# LSTM (Long Short-Term Memory): a type of recurrent neural network (RNN) layer that is useful for learning long-term dependencies
from tensorflow.keras.layers import LSTM  # for modeling sequential data with long-term dependencies

# Dense: a fully connected layer that is used for outputting results from the model
from tensorflow.keras.layers import Dense  # for outputting results from the model

# Bidirectional: a wrapper layer that allows an RNN layer to process input sequences in both forward and backward directions
from tensorflow.keras.layers import Bidirectional  # for modeling sequential data with dependencies in both directions

In [63]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [64]:
# Create a copy of the original dataset `X` to ensure the original data remains unchanged.
# This is useful for data preprocessing steps where modifications are needed without altering the original data.
messages = X.copy()

In [65]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [66]:
# Reset the index of the DataFrame `messages` and update it in place.
# This is useful to ensure the DataFrame has a clean, sequential index after any filtering or modifications.
# Setting `inplace=True` means the operation is performed on the DataFrame directly, rather than returning a new DataFrame.
messages.reset_index(inplace=True)

In [67]:
# Importing necessary modules from the NLTK (Natural Language Toolkit) library and regular expressions:
# nltk: A leading platform for building Python programs to work with human language data.
import nltk

# re: Provides support for regular expressions, allowing for complex string manipulation and pattern matching.
import re

# stopwords: Import the list of common stopwords in the English language from the NLTK corpus.
# Stopwords are commonly used words (such as 'the', 'is', 'in') that are often removed from text data to improve text analysis.
from nltk.corpus import stopwords

In [68]:
# Download the list of stopwords from the NLTK corpus.
# Stopwords are common words that are typically removed from text data during preprocessing to improve analysis.
# This step ensures that the stopwords list is available for use in text processing tasks.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JATIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:
### Dataset Preprocessing

# Import the PorterStemmer for stemming words (reducing them to their root form)
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# Initialize an empty list to hold the cleaned and processed text data
corpus = []

# Loop through each message in the dataset
for i in range(0, len(messages)):
    # Remove non-alphabetical characters from the 'title' field and replace them with spaces
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    
    # Convert all characters in the review to lowercase
    review = review.lower()
    
    # Split the review into individual words
    review = review.split()
    
    # Stem each word in the review and remove stopwords
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    
    # Join the processed words back into a single string
    review = ' '.join(review)
    
    # Append the cleaned review to the corpus
    corpus.append(review)

In [70]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [71]:
# Create a list of one-hot representations for each word in the corpus
onehot_repr = [one_hot(words, voc_size) for words in corpus]
  # Iterate over each word (words) in the corpus
  # one_hot function encodes each word into a numerical representation (one-hot vector)
  # voc_size is the size of the vocabulary, which determines the length of the one-hot vector
  # The resulting list (onehot_repr) contains one-hot representations for each word in the corpus

In [72]:
onehot_repr

[[3135, 1263, 2596, 748, 2703, 3221, 4088, 660, 2083, 1570],
 [1812, 1139, 190, 4278, 4245, 241, 4763],
 [3231, 4510, 2219, 2867],
 [4923, 3632, 3017, 4624, 2638, 2476],
 [2697, 4245, 49, 1842, 446, 4523, 4245, 2902, 733, 4383],
 [4816,
  853,
  4823,
  1275,
  4459,
  2712,
  1510,
  1684,
  2362,
  2460,
  4976,
  175,
  4174,
  2397,
  4763],
 [1810, 1744, 4210, 2258, 2777, 3687, 897, 4463, 2025, 1004, 3837],
 [4601, 1550, 88, 3574, 1909, 4430, 2712, 2498, 2025, 1004, 3837],
 [913, 1356, 1393, 4308, 123, 1150, 2389, 4231, 2712, 1482],
 [1076, 3204, 1694, 858, 1417, 4746, 2866, 1364],
 [4353, 940, 4676, 2250, 3070, 2940, 3820, 2134, 1459, 534, 2152],
 [4624, 3155, 2703, 1150, 2712, 1909],
 [1223, 3081, 854, 3953, 4018, 4515, 805, 1702, 542],
 [3028, 3515, 3885, 4843, 956, 848, 769, 2025, 1004, 3837],
 [51, 2014, 2032, 4316, 1695, 2025, 1004, 3837],
 [442, 4687, 4656, 2916, 4460, 1525, 1028, 411, 4762, 3524],
 [3915, 2524, 1139],
 [103, 1488, 3256, 1090, 2712, 1712, 327, 4763],
 [3446

### Embedding Representation

In [73]:
# Set the maximum length of a sentence to 20
sent_length = 20

# Pad the one-hot representations of documents to a fixed length (sent_length)
# using zeros at the beginning of each sequence (padding='pre')
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

# Print the padded sequences
print(embedded_docs)

[[   0    0    0 ...  660 2083 1570]
 [   0    0    0 ... 4245  241 4763]
 [   0    0    0 ... 4510 2219 2867]
 ...
 [   0    0    0 ... 2025 1004 3837]
 [   0    0    0 ... 4139  770  902]
 [   0    0    0 ... 1353  772 1665]]


In [74]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3135,
       1263, 2596,  748, 2703, 3221, 4088,  660, 2083, 1570])

In [75]:
## Creating model

# Define the number of features in the embedding vector
embedding_vector_features = 40

# Create a Sequential model
model = Sequential()

# Add an Embedding layer to the model
# This layer converts categorical data (words) into dense vectors of fixed size
# voc_size is the size of the vocabulary, embedding_vector_features is the number of features in the embedding vector
# input_length is the length of the input sequences (sentences)
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))

# Add a Bidirectional LSTM layer to the model
# This layer models sequential data with dependencies in both forward and backward directions
# The LSTM layer has 100 units (neurons)
model.add(Bidirectional(LSTM(100)))

# Add a Dense layer to the model
# This layer outputs a single value (probability) with a sigmoid activation function
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a binary cross-entropy loss function, Adam optimizer, and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
len(embedded_docs),y.shape

(18285, (18285,))

In [77]:
import numpy as np

# Convert the padded sequences (embedded_docs) into a NumPy array
# to prepare it for training the model
X_final = np.array(embedded_docs)

# Convert the target variable (y) into a NumPy array
# to prepare it for training the model
y_final = np.array(y)

In [78]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [80]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 23s 76ms/step - accuracy: 0.7946 - loss: 0.4047 - val_accuracy: 0.9148 - val_loss: 0.1970
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - accuracy: 0.9527 - loss: 0.1272 - val_accuracy: 0.9138 - val_loss: 0.2021
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 75ms/step - accuracy: 0.9664 - loss: 0.0916 - val_accuracy: 0.9143 - val_loss: 0.2282
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 75ms/step - accuracy: 0.9781 - loss: 0.0666 - val_accuracy: 0.9064 - val_loss: 0.2770
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 74ms/step - accuracy: 0.9870 - loss: 0.0476 - val_accuracy: 0.9080 - val_loss: 0.2792
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - accuracy: 0.9919 - loss: 0.0312 - val_accuracy: 0.9109 - val_loss: 0.3308
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 75ms/step - accuracy: 0.9934 - loss: 0.0235 - val_accuracy: 0.9049 - val_loss: 0.4121
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 76ms/step - accuracy: 0.9968 - loss: 0.0141 - 

### Performance Metrics And Accuracy

In [81]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step


In [82]:
# Convert the predicted probabilities to binary class labels
# by thresholding at 0.5
y_pred = np.where(y_pred > 0.5, 1, 0)

In [83]:
from sklearn.metrics import confusion_matrix

In [84]:
confusion_matrix(y_test,y_pred)

array([[3160,  259],
       [ 318, 2298]], dtype=int64)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9043910521955261

In [86]:
from sklearn.metrics import classification_report

In [87]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3478
           1       0.88      0.90      0.89      2557

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

